In [170]:
import time
import pandas as pd
import os
import collections
import numpy as np
import statsmodels.api as sm
import pylogit
import pickle
from statsmodels.iolib.summary2 import summary_col
from GetProcessMNL_LR import PreProcessMNL, OLS_Model
import matplotlib.pyplot as plt
%matplotlib inline

# Combined Linear Regression

In [2]:
i = 0
Airport = [['JFK','FLL'],['JFK','LAX'],['IAH','BOS'],
           ['FLL','JFK'],['LAX','JFK'],['BOS','IAH']]
for pair in Airport:
    i += 1
    if i == 1:
        PostMNL = PreProcessMNL(pair[0], pair[1], 2013)
        LR_Data, MNL_Data = PostMNL.GetLR_Data(OneMetric = True)
    else:
        PostMNL = PreProcessMNL(pair[0], pair[1], 2013)
        LR_temp, MNL_temp = PostMNL.GetLR_Data(OneMetric = True)
        LR_Data = LR_Data.append(LR_temp)
        MNL_Data = MNL_Data.append(MNL_temp)
LR_Data = LR_Data.reset_index(drop = 1)
MNL_Data = MNL_Data.reset_index(drop = 1)
print(i)

6


In [3]:
LR_Data.to_csv(os.getcwd() + '/LR/LR_SixPairs_OneMetric.csv', index = False)
MNL_Data.to_csv(os.getcwd() + '/LR/MNL_SixPairs_Combined.csv', index = False)

In [3]:
LR_Data.Efficiency = LR_Data.Efficiency * 100

In [4]:
from patsy.contrasts import Treatment

In [147]:
MNL_Data.Pred_Eff.

0.46781513541807196

In [175]:
LR_model = sm.OLS.from_formula('np.log(Efficiency) ~ TS_mean + Rain_mean + C(OD_Clust) + C(Season) + BusyHour + MIT_Str', data=LR_Data).fit()
print LR_model.summary()

                            OLS Regression Results                            
Dep. Variable:     np.log(Efficiency)   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     1476.
Date:                Mon, 27 Feb 2017   Prob (F-statistic):               0.00
Time:                        22:03:29   Log-Likelihood:                -16780.
No. Observations:               32409   AIC:                         3.364e+04
Df Residuals:                   32369   BIC:                         3.397e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------
Intercept             

In [174]:
# from statsmodels.stats.outliers_influence import summary_table
# from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [250]:
from scipy import stats
def transform_exog_to_model(fit, exog):
    transform=True
    self=fit
    
    # The following is lifted straight from statsmodels.base.model.Results.predict()
    if transform and hasattr(self.model, 'formula') and exog is not None:
        from patsy import dmatrix
        exog = dmatrix(self.model.data.orig_exog.design_info.builder,
                       exog)

    if exog is not None:
        exog = np.asarray(exog)
        if exog.ndim == 1 and (self.model.exog.ndim == 1 or
                               self.model.exog.shape[1] == 1):
            exog = exog[:, None]
        exog = np.atleast_2d(exog)  # needed in count model shape[1]
    
    # end lifted code
    return exog

def wls_prediction_std(res, exog=None, weights=None, alpha=0.05):
    covb = res.cov_params()
    if exog is None:
        exog = res.model.exog
        predicted = res.fittedvalues
        if weights is None:
            weights = res.model.weights
    else:
        # Using res.predict() is less hacky than using res.model.predict()
        predicted = res.predict(exog)

        if covb.shape[1] != exog.shape[1]:
            # raise ValueError('wrong shape of exog') # Nope
            # Instead of panicking/raising ValueError in this case, transform 
            # the exog appropriately so that the interval calculations below can
            # function
            exog = transform_exog_to_model(res, exog)

        if weights is None:
            weights = 1.
        else:
            weights = np.asarray(weights)
            if len(weights) > 1 and len(weights) != exog.shape[0]:
                raise ValueError('weights and exog do not have matching shape')

    # The rest of this is unaltered

    if weights is None:
        weights = res.model.weights


    # full covariance:
    #predvar = res3.mse_resid + np.diag(np.dot(X2,np.dot(covb,X2.T)))
    # predication variance only
    predvar = res.mse_resid/weights + (exog * np.dot(covb, exog.T).T).sum(1)
    predstd = np.sqrt(predvar)
    tppf = stats.t.isf(alpha/2., res.df_resid)
    interval_u = predicted + tppf * predstd
    interval_l = predicted - tppf * predstd
    return predstd, interval_l, interval_u

In [251]:
PredStd, iv_l, iv_u = wls_prediction_std(LR_model, exog = MNL_Data)

In [252]:
MNL_Pred = LR_model.predict(exog=MNL_Data)
MNL_Data['Pred_Eff'] = np.exp(MNL_Pred)
MNL_Data['Pred_Eff_Std'] = PredStd
MNL_Data['Pred_Eff_CI_up'] = np.exp(iv_u)
MNL_Data['Pred_Eff_CI_low'] = np.exp(iv_l)

In [237]:
print MNL_Data['Pred_Eff'].mean()
print MNL_Data['Pred_Eff'].max()
print MNL_Data['Pred_Eff'].min()

3.94057886006
26.5146853141
0.736305518061


In [255]:
print MNL_Data['Pred_Eff_Std'].mean()
print MNL_Data['Pred_Eff_Std'].max()
print MNL_Data['Pred_Eff_Std'].min()

0.407501768397
0.41974182067
0.406399463871


In [238]:
print MNL_Data['Pred_Eff_CI_up'].mean()
print MNL_Data['Pred_Eff_CI_up'].max()
print MNL_Data['Pred_Eff_CI_up'].min()

8.77516495621
59.8338690179
1.63310392006


In [239]:
print MNL_Data['Pred_Eff_CI_low'].mean()
print MNL_Data['Pred_Eff_CI_low'].max()
print MNL_Data['Pred_Eff_CI_low'].min()

1.76960310617
11.7496753735
0.331972637668


### RFR

In [140]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le = LabelEncoder()
le.fit(LR_Data['OD_Clust'])
# enc = OneHotEncoder()
# enc.fit(le.transform(LR_Data['OD_Clust']).reshape(-1,1))  
LR_Data['OD_Clust_numerical'] = le.transform(LR_Data['OD_Clust']) 
MNL_Data['OD_Clust_numerical'] = le.transform(MNL_Data['OD_Clust']) 
# np.hstack((LR_Data[['TS_mean', 'Rain_mean', 'Hail_mean', 
#                                  'Precipitation_mean', 'Shower_mean', 
#                                  'Ice_mean', 'Squall_mean','Season','MIT_Str',
#                                  'BusyHour','OD_Clust_numerical']].values,pd.get_dummies(LR_Data['OD_Clust']).values)).shape

rf_class = RandomForestRegressor(max_depth = 25, random_state=101, n_estimators=200, max_features=None)
rf_model = rf_class.fit(LR_Data[['TS_mean', 'Rain_mean', 'Hail_mean', 
                                 'Precipitation_mean', 'Shower_mean', 
                                 'Ice_mean', 'Squall_mean','Season','MIT_Str',
                                 'BusyHour','OD_Clust_numerical']], LR_Data['Efficiency'])
rf_model.feature_importances_
MNL_Pred = rf_model.predict(MNL_Data[['TS_mean', 'Rain_mean', 'Hail_mean', 
                                 'Precipitation_mean', 'Shower_mean', 
                                 'Ice_mean', 'Squall_mean','Season','MIT_Str',
                                 'BusyHour','OD_Clust_numerical']])
MNL_Data['Pred_Eff'] = MNL_Pred

# MNL

In [246]:
BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

# BaseSpec['intercept'] = [0,1,2,3,4]
# BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']
# BaseSpec['Efficiency'] = [0,1,2,3,4]
# BaseName['Efficiency'] = ['Inefficiency_R0','Inefficiency_R1','Inefficiency_R2','Inefficiency_R3','Inefficiency_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0,1,2,3,4]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3','Spring_R4']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R4']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
# BaseSpec['TS_mean'] = [[0,1,2,3,4,5]]
# BaseName['TS_mean'] = ['Thunderstorm']

# # BaseSpec['Hail_mean'] = [[0,1,2,3,4]]
# # BaseName['Hail_mean'] = ['Hail']

# # BaseSpec['Precipitation_mean'] = [[0,1,2,3,4]]
# # BaseName['Precipitation_mean'] = ['Precipitation']

# BaseSpec['Rain_mean'] = [[0,1,2,3,4,5]]
# BaseName['Rain_mean'] = ['Rain']

# # BaseSpec['Shower_mean'] = [[0,1,2,3,4,5]]
# # BaseName['Shower_mean'] = ['Shower']

# # BaseSpec['Ice_sum'] = [[0,1,2,3,4,5]]
# # BaseName['Ice_sum'] = ['Ice']

# # BaseSpec['Squall_mean'] = [[0,1,2,3,4,5]]
# # BaseName['Squall_mean'] = ['Squall']

# BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4,5]]
# BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# # BaseSpec['MIT_Count'] = [[0,1,2,3,4,5]]
# # BaseName['MIT_Count'] = ['MIT_Count']

# # BaseSpec['MITVAL'] = [[0,1,2,3,4,5]]
# # BaseName['MITVAL'] = ['MIT_Value']

# # BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4,5]]
# # BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

# BaseSpec['MIT_Str'] = [[0,1,2,3,4,5]]
# BaseName['MIT_Str'] = ['MIT_Impact']

BaseSpec['Pred_Eff'] = [[0,1,2,3,4,5]]
BaseName['Pred_Eff'] = ['PredIneff']

BaseSpec['Pred_Eff_Std'] = [[0,1,2,3,4,5]]
BaseName['Pred_Eff_Std'] = ['PredIneff_Std']

# BaseSpec['Pred_Eff_CI_low'] = [[0,1,2,3,4,5]]
# BaseName['Pred_Eff_CI_low'] = ['Pred_Eff_CI_low']

# BaseSpec['Pred_Eff_CI_up'] = [[0,1,2,3,4,5]]
# BaseName['Pred_Eff_CI_up'] = ['Pred_Eff_CI_up']


N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = MNL_Data[MNL_Data.OD == 'IAH_BOS'], alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

Log-likelihood at zero: -2,981.4878
Initial Log-likelihood: -2,981.4878
Estimation Time: 0.15 seconds.
Final log-likelihood: -2,173.5367
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                1,664
Model:             Multinomial Logit Model   Df Residuals:                    1,643
Method:                                MLE   Df Model:                           21
Date:                     Mon, 27 Feb 2017   Pseudo R-squ.:                   0.271
Time:                             22:20:09   Pseudo R-bar-squ.:               0.264
converged:                            True   Log-Likelihood:             -2,173.537
                                             LL-Null:                    -2,981.488
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------

In [256]:
# BOS - IAH
BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

# BaseSpec['intercept'] = [0,1,2,3,4]
# BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0,1,2,3,4]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3','Spring_R4']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,3,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R3','Fall_R4']
    
# BaseSpec['TS_mean'] = [[0,1,2,3,4,5]]
# BaseName['TS_mean'] = ['Thunderstorm']

# BaseSpec['Rain_mean'] = [[0,1,2,3,4,5]]
# BaseName['Rain_mean'] = ['Rain']

# BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4,5]]
# BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4,5]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3,4,5]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4,5]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

# BaseSpec['MIT_Str'] = [[1,2,3,4,5]]
# BaseName['MIT_Str'] = ['MIT_Impact']

BaseSpec['Pred_Eff'] = [[0,1,2,3,4,5]]
BaseName['Pred_Eff'] = ['PredIneff']

BaseSpec['Pred_Eff_Std'] = [[0,1,2,3,4,5]]
BaseName['Pred_Eff_Std'] = ['PredIneff_Std']

# BaseSpec['Pred_Eff_CI_low'] = [[0,1,2,3,4,5]]
# BaseName['Pred_Eff_CI_low'] = ['Pred_Eff_CI_low']

# BaseSpec['Pred_Eff_CI_up'] = [[0,1,2,3,4,5]]
# BaseName['Pred_Eff_CI_up'] = ['Pred_Eff_CI_up']


N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = MNL_Data[MNL_Data.OD == 'BOS_IAH'], alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

Log-likelihood at zero: -3,103.3274
Initial Log-likelihood: -3,103.3274
Estimation Time: 0.15 seconds.
Final log-likelihood: -2,154.4719
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                1,732
Model:             Multinomial Logit Model   Df Residuals:                    1,710
Method:                                MLE   Df Model:                           22
Date:                     Mon, 27 Feb 2017   Pseudo R-squ.:                   0.306
Time:                             22:41:54   Pseudo R-bar-squ.:               0.299
converged:                            True   Log-Likelihood:             -2,154.472
                                             LL-Null:                    -3,103.327
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------

# Logistic Regression

#### IAH -> BOS

In [2]:
DEP = 'IAH'
ARR = 'BOS'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3,4]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0,1,2,3,4]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3','Spring_R4']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R4']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3,4,5]]
BaseName['TS_mean'] = ['Thunderstorm']

# BaseSpec['Hail_mean'] = [[0,1,2,3,4]]
# BaseName['Hail_mean'] = ['Hail']

# BaseSpec['Precipitation_mean'] = [[0,1,2,3,4]]
# BaseName['Precipitation_mean'] = ['Precipitation']

BaseSpec['Rain_mean'] = [[0,1,2,3,4,5]]
BaseName['Rain_mean'] = ['Rain']

# BaseSpec['Shower_mean'] = [[0,1,2,3,4,5]]
# BaseName['Shower_mean'] = ['Shower']

# BaseSpec['Ice_sum'] = [[0,1,2,3,4,5]]
# BaseName['Ice_sum'] = ['Ice']

# BaseSpec['Squall_mean'] = [[0,1,2,3,4,5]]
# BaseName['Squall_mean'] = ['Squall']

BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4,5]]
BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4,5]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3,4,5]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4,5]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

# BaseSpec['MIT_Str'] = [[0,1,2,3,4,5]]
# BaseName['MIT_Str'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

# pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

Log-likelihood at zero: -2,981.4878
Initial Log-likelihood: -2,981.4878
Estimation Time: 0.15 seconds.
Final log-likelihood: -1,989.3664
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                1,664
Model:             Multinomial Logit Model   Df Residuals:                    1,637
Method:                                MLE   Df Model:                           27
Date:                     Fri, 10 Feb 2017   Pseudo R-squ.:                   0.333
Time:                             16:15:54   Pseudo R-bar-squ.:               0.324
converged:                            True   Log-Likelihood:             -1,989.366
                                             LL-Null:                    -2,981.488
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------

E:\Anaconda\Anaconda\lib\site-packages\scipy\optimize\_minimize.py:385: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


#### BOS_IAH

In [34]:
DEP = 'BOS'
ARR = 'IAH'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3,4]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0,1,2,3,4]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3','Spring_R4']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,3,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R3','Fall_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3,4,5]]
BaseName['TS_mean'] = ['Thunderstorm']

BaseSpec['Rain_mean'] = [[0,1,2,3,4,5]]
BaseName['Rain_mean'] = ['Rain']

BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4,5]]
BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4,5]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3,4,5]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4,5]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

# BaseSpec['MIT_Str'] = [[1,2,3,4,5]]
# BaseName['MIT_Str'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

# pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -3,103.3274
Initial Log-likelihood: -3,103.3274
Estimation Time: 0.14 seconds.
Final log-likelihood: -2,001.5202
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                1,732
Model:             Multinomial Logit Model   Df Residuals:                    1,704
Method:                                MLE   Df Model:                           28
Date:                     Tue, 07 Feb 2017   Pseudo R-squ.:                   0.355
Time:                             10:13:41   Pseudo R-bar-squ.:               0.346
converged:                            True   Log-Likelihood:             -2,001.520
                                             LL-Null:                    -3,103.327
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### FLL-JFK

In [36]:
DEP = 'FLL'
ARR = 'JFK'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2']

BaseSpec['Morning'] = [0,1,2]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2']

BaseSpec['Spring'] = [0,1,2]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2']

BaseSpec['Summer'] = [0,1,2]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2']

BaseSpec['Fall'] = [0,1,2]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3]]
BaseName['TS_mean'] = ['Thunderstorm']

# BaseSpec['Hail_mean'] = [[0,1,2]]
# BaseName['Hail_mean'] = ['Hail']
# BaseSpec['Precipitation_mean'] = [[0,1,2,3,4]]
# BaseName['Precipitation_mean'] = ['Precipitation']

BaseSpec['Rain_mean'] = [[0,1,2,3]]
BaseName['Rain_mean'] = ['Rain']

# BaseSpec['Squall_mean'] = [[0,1,2,3]]
# BaseName['Squall_mean'] = ['Squall']

# BaseSpec['Std_Wind_Dist'] = [[0,1,2,3]]
# BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

BaseSpec['MIT_Str'] = [[0,1,2,3]]
BaseName['MIT_Str'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -5,528.5419
Initial Log-likelihood: -5,528.5419
Estimation Time: 0.18 seconds.
Final log-likelihood: -1,795.0199
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                3,988
Model:             Multinomial Logit Model   Df Residuals:                    3,970
Method:                                MLE   Df Model:                           18
Date:                     Tue, 07 Feb 2017   Pseudo R-squ.:                   0.675
Time:                             10:14:25   Pseudo R-bar-squ.:               0.672
converged:                           False   Log-Likelihood:             -1,795.020
                                             LL-Null:                    -5,528.542
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### JFK-FLL

In [37]:
DEP = 'JFK'
ARR = 'FLL'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3,4]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0,1,2,3,4]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3','Spring_R4']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,3,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R3','Fall_R4']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3,4,5]]
BaseName['TS_mean'] = ['Thunderstorm']

# BaseSpec['Hail_mean'] = [[0,1,2,3,4]]
# BaseName['Hail_mean'] = ['Hail']

# BaseSpec['Precipitation_mean'] = [[0,1,2,3,4]]
# BaseName['Precipitation_mean'] = ['Precipitation']

BaseSpec['Rain_mean'] = [[0,1,2,3,4,5]]
BaseName['Rain_mean'] = ['Rain']

# BaseSpec['Squall_mean'] = [[0,1,2,3,4,5]]
# BaseName['Squall_mean'] = ['Squall']

# BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4,5]]
# BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4,5]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3,4,5]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4,5]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

BaseSpec['MIT_Str'] = [[0,1,2,3,4,5]]
BaseName['MIT_Str'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -7,204.6648
Initial Log-likelihood: -7,204.6648
Estimation Time: 0.44 seconds.
Final log-likelihood: -2,332.4358
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                4,021
Model:             Multinomial Logit Model   Df Residuals:                    3,993
Method:                                MLE   Df Model:                           28
Date:                     Tue, 07 Feb 2017   Pseudo R-squ.:                   0.676
Time:                             10:14:47   Pseudo R-bar-squ.:               0.672
converged:                            True   Log-Likelihood:             -2,332.436
                                             LL-Null:                    -7,204.665
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### JFK-LAX

In [50]:
DEP = 'JFK'
ARR = 'LAX'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3,4]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0,1,2,3,4]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3','Spring_R4']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,3,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R3','Fall_R4']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3,4,5]]
BaseName['TS_mean'] = ['Thunderstorm']

# BaseSpec['Hail_mean'] = [[0,1,2,3]]
# BaseName['Hail_mean'] = ['Hail']
# BaseSpec['Precipitation_mean'] = [[0,1,2,3,4]]
# BaseName['Precipitation_mean'] = ['Precipitation']

BaseSpec['Rain_mean'] = [[0,1,2,3,4,5]]
BaseName['Rain_mean'] = ['Rain']

BaseSpec['Squall_mean'] = [[0,1,2,3,4,5]]
BaseName['Squall_mean'] = ['Squall']

BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4,5]]
BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3,4]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

BaseSpec['MIT_Str'] = [[0,1,2,3,4,5]]
BaseName['MIT_Str'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

# pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -19,058.9455
Initial Log-likelihood: -19,058.9455
Estimation Time: 2.53 seconds.
Final log-likelihood: -15,250.6962
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:               10,637
Model:             Multinomial Logit Model   Df Residuals:                   10,607
Method:                                MLE   Df Model:                           30
Date:                     Mon, 06 Feb 2017   Pseudo R-squ.:                   0.200
Time:                             21:51:16   Pseudo R-bar-squ.:               0.198
converged:                           False   Log-Likelihood:            -15,250.696
                                             LL-Null:                   -19,058.945
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------

#### LAX-JFK

In [49]:
DEP = 'LAX'
ARR = 'JFK'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3']

BaseSpec['Morning'] = [0,1,2,3]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3']

BaseSpec['Spring'] = [0,1,2,3]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3']

BaseSpec['Summer'] = [0,1,2,3]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3']

BaseSpec['Fall'] = [0,1,2,3]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R3']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3,4]]
BaseName['TS_mean'] = ['ThunderStorm']

# BaseSpec['Hail_mean'] = [[0,1,2,3,4]]
# BaseName['Hail_mean'] = ['Hail']

BaseSpec['Rain_mean'] = [[0,1,2,3,4]]
BaseName['Rain_mean'] = ['Rain']

BaseSpec['Squall_mean'] = [[0,1,2,3,4]]
BaseName['Squall_mean'] = ['Squall']

BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4]]
BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3,4]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

BaseSpec['MIT_Str'] = [[0,1,2,3,4]]
BaseName['MIT_Str'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

# pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -16,685.0428
Initial Log-likelihood: -16,685.0428
Estimation Time: 2.41 seconds.
Final log-likelihood: -13,813.6127
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:               10,367
Model:             Multinomial Logit Model   Df Residuals:                   10,342
Method:                                MLE   Df Model:                           25
Date:                     Mon, 06 Feb 2017   Pseudo R-squ.:                   0.172
Time:                             21:23:02   Pseudo R-bar-squ.:               0.171
converged:                           False   Log-Likelihood:            -13,813.613
                                             LL-Null:                   -16,685.043
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------

### DCA -> ORD

In [9]:
DEP = 'DCA'
ARR = 'ORD'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3,4]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3','ASC_R4']

BaseSpec['Morning'] = [0,1,2,3,4]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3','MorningFlight_R4']

BaseSpec['Spring'] = [0]
BaseName['Spring'] = ['Spring_R0']

BaseSpec['Summer'] = [0,1,2,3,4]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3','Summer_R4']

BaseSpec['Fall'] = [0,1,2,3,4]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2','Fall_R3','Fall_R4']

BaseSpec['CA'] = [0,1,2,3,4]
BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3,4]]
BaseName['TS_mean'] = ['Thunderstorm']

BaseSpec['Rain_mean'] = [[0,1,2,3,4]]
BaseName['Rain_mean'] = ['Rain']

BaseSpec['Std_Wind_Dist'] = [[0,1,2,3,4]]
BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[4]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3,4]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

# BaseSpec['MIT_Impact'] = [[4]]
# BaseName['MIT_Impact'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print(Result1.as_csv)

pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -13,155.0980
Initial Log-likelihood: -13,155.0980
Estimation Time: 1.95 seconds.
Final log-likelihood: -1,448.4786


<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                7,342
Model:             Multinomial Logit Model   Df Residuals:                    7,312
Method:                                MLE   Df Model:                           30
Date:                     Thu, 19 Jan 2017   Pseudo R-squ.:                   0.890
Time:                             20:41:23   Pseudo R-bar-squ.:               0.888
converged:                            True   Log-Likelihood:             -1,448.479
                                             LL-Null:                   -13,155.098
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
ASC_R0               5.5406      1.247      4.444      0.000         3.097    

### ORD -> DCA

In [10]:
DEP = 'ORD'
ARR = 'DCA'
Year = 2013
PostMNL = PreProcessMNL(DEP, ARR, Year)
modelset= PostMNL.GetMNL_Data()

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3']

BaseSpec['Morning'] = [0,1,2,3]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3']

BaseSpec['Spring'] = [0,1,2]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2']

BaseSpec['Summer'] = [0,1,2,3]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3']

BaseSpec['Fall'] = [0,1,2]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2']

# BaseSpec['CA'] = [0,1,2,3,4]
# BaseName['CA'] = ['United_R0','United_R1','United_R2','United_R3','United_R4']
    
BaseSpec['TS_mean'] = [[0,1,2,3]]
BaseName['TS_mean'] = ['Thunderstorm']

# BaseSpec['Hail_mean'] = [[0,1,2,3]]
# BaseName['Hail_mean'] = ['Hail']
# BaseSpec['Precipitation_mean'] = [[0,1,2,3,4]]
# BaseName['Precipitation_mean'] = ['Precipitation']

BaseSpec['Rain_mean'] = [[0,1,2,3]]
BaseName['Rain_mean'] = ['Rain']

BaseSpec['Std_Wind_Dist'] = [[0,1,2,3]]
BaseName['Std_Wind_Dist'] = ['Std_Wind_Dist']

# BaseSpec['MIT_Count'] = [[0,1,2,3,4]]
# BaseName['MIT_Count'] = ['MIT_Count']

# BaseSpec['MITVAL'] = [[0,1,2,3]]
# BaseName['MITVAL'] = ['MIT_Value']

# BaseSpec['DURATION_HRS_TOT'] = [[0,1,2,3]]
# BaseName['DURATION_HRS_TOT'] = ['MIT_Duration']

# BaseSpec['MIT_Impact'] = [[0,1,2,3]]
# BaseName['MIT_Impact'] = ['MIT_Impact']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
print Result1.as_csv

pickle.dump(MNL_Model, open( os.getcwd() + '/MNL/Estimation/MNL_' + DEP + '_' + ARR+'.p', "wb" ) )

E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Log-likelihood at zero: -11,756.9440
Initial Log-likelihood: -11,756.9440
Estimation Time: 1.34 seconds.
Final log-likelihood: -1,164.1833


<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                      CHOICE   No. Observations:                7,305
Model:             Multinomial Logit Model   Df Residuals:                    7,283
Method:                                MLE   Df Model:                           22
Date:                     Thu, 19 Jan 2017   Pseudo R-squ.:                   0.901
Time:                             20:42:02   Pseudo R-bar-squ.:               0.899
converged:                           False   Log-Likelihood:             -1,164.183
                                             LL-Null:                   -11,756.944
                       coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
ASC_R0              -0.6137      2.601     -0.236      0.813        -5.712    

# Composite Method

In [2]:
IAH_BOS = OLS_Model('IAH','BOS',2013, OneMetric = True)

In [3]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + MIT_Str'
mod1 = IAH_BOS.fitModel(fo1)
print mod1.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.610
Model:                            OLS   Adj. R-squared:                  0.607
Method:                 Least Squares   F-statistic:                     258.1
Date:                Fri, 10 Feb 2017   Prob (F-statistic):               0.00
Time:                        16:19:36   Log-Likelihood:                -2882.4
No. Observations:                1664   AIC:                             5787.
Df Residuals:                    1653   BIC:                             5846.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           8.5453      0.159     

In [42]:
fo1 = 'Efficiency ~ TS_mean + Rain_mean + C(ClustID) + C(season)'
mod1 = IAH_BOS.fitModel(fo1)
print mod1.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.626
Method:                 Least Squares   F-statistic:                     279.1
Date:                Tue, 07 Feb 2017   Prob (F-statistic):               0.00
Time:                        10:19:13   Log-Likelihood:                -2842.1
No. Observations:                1664   AIC:                             5706.
Df Residuals:                    1653   BIC:                             5766.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           8.0049      0.166     

In [4]:
BOS_IAH = OLS_Model('BOS','IAH',2013)

In [11]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour'
mod2 = BOS_IAH.fitModel(fo1)
print mod2.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     313.0
Date:                Fri, 10 Feb 2017   Prob (F-statistic):               0.00
Time:                        16:21:13   Log-Likelihood:                -2789.6
No. Observations:                1732   AIC:                             5601.
Df Residuals:                    1721   BIC:                             5661.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           5.8102      0.113     

In [44]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour + MIT_Str'
mod2 = BOS_IAH.fitModel(fo1)
print mod2.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.650
Model:                            OLS   Adj. R-squared:                  0.648
Method:                 Least Squares   F-statistic:                     290.9
Date:                Tue, 07 Feb 2017   Prob (F-statistic):               0.00
Time:                        10:19:18   Log-Likelihood:                -2777.0
No. Observations:                1732   AIC:                             5578.
Df Residuals:                    1720   BIC:                             5643.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           5.5624      0.115     

In [12]:
FLL_JFK = OLS_Model('FLL','JFK',2013)

In [16]:
fo1 = 'Efficiency ~ C(ClustID) + C(season) + BusyHour + MIT_Str'
mod3 = FLL_JFK.fitModel(fo1)
print mod3.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     580.6
Date:                Fri, 10 Feb 2017   Prob (F-statistic):               0.00
Time:                        16:22:13   Log-Likelihood:                -9960.5
No. Observations:                3988   AIC:                         1.994e+04
Df Residuals:                    3979   BIC:                         2.000e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept          20.0589      0.370     

In [46]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour + MIT_Str'
mod3 = FLL_JFK.fitModel(fo1)
print mod3.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.537
Method:                 Least Squares   F-statistic:                     514.9
Date:                Tue, 07 Feb 2017   Prob (F-statistic):               0.00
Time:                        10:19:27   Log-Likelihood:                -9962.6
No. Observations:                3988   AIC:                         1.995e+04
Df Residuals:                    3978   BIC:                         2.001e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept          19.7424      0.371     

In [17]:
JFK_FLL = OLS_Model('JFK','FLL',2013)

In [19]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour + MIT_Str + Rain_mean'
mod4 = JFK_FLL.fitModel(fo1)
print mod4.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     1037.
Date:                Fri, 10 Feb 2017   Prob (F-statistic):               0.00
Time:                        16:22:45   Log-Likelihood:                -7707.8
No. Observations:                4021   AIC:                         1.544e+04
Df Residuals:                    4008   BIC:                         1.552e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           9.9627      0.178     

In [48]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour + MIT_Str'
mod4 = JFK_FLL.fitModel(fo1)
print mod4.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                     1136.
Date:                Tue, 07 Feb 2017   Prob (F-statistic):               0.00
Time:                        10:19:32   Log-Likelihood:                -7701.7
No. Observations:                4021   AIC:                         1.543e+04
Df Residuals:                    4009   BIC:                         1.550e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           9.6467      0.178     

In [20]:
JFK_LAX = OLS_Model('JFK','LAX',2013)

In [24]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour'
mod5 = JFK_LAX.fitModel(fo1)
print mod5.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.499
Model:                            OLS   Adj. R-squared:                  0.498
Method:                 Least Squares   F-statistic:                     1057.
Date:                Fri, 10 Feb 2017   Prob (F-statistic):               0.00
Time:                        16:23:28   Log-Likelihood:                -16790.
No. Observations:               10637   AIC:                         3.360e+04
Df Residuals:                   10626   BIC:                         3.368e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           3.5707      0.037     

In [25]:
LAX_JFK = OLS_Model('LAX','JFK',2013)

In [31]:
fo1 = 'Efficiency ~ TS_mean + C(ClustID) + C(season) + BusyHour + MIT_Str'
mod6 = LAX_JFK.fitModel(fo1)
print mod6.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.278
Model:                            OLS   Adj. R-squared:                  0.277
Method:                 Least Squares   F-statistic:                     398.7
Date:                Fri, 10 Feb 2017   Prob (F-statistic):               0.00
Time:                        16:25:10   Log-Likelihood:                -19956.
No. Observations:               10367   AIC:                         3.993e+04
Df Residuals:                   10356   BIC:                         4.001e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           3.6900      0.049     

In [51]:
fo1 = 'Efficiency ~ TS_mean + Rain_mean + C(ClustID) + C(season) + BusyHour + MIT_Str'
mod6 = LAX_JFK.fitModel(fo1)
print mod6.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.325
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     453.5
Date:                Tue, 07 Feb 2017   Prob (F-statistic):               0.00
Time:                        10:20:10   Log-Likelihood:                -19606.
No. Observations:               10367   AIC:                         3.924e+04
Df Residuals:                   10355   BIC:                         3.932e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
Intercept           3.3948      0.049     

In [52]:
Combined_Results = summary_col([mod1,mod2,mod3,mod4,mod5,mod6],stars=True,float_format='%0.2f',
                  model_names=['IAH BOS','BOS IAH','FLL JFK', 'JFK FLL','JFK LAX', 'LAX FLL'],
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)})
# .as_latex()

E:\Anaconda\Anaconda\lib\site-packages\statsmodels\iolib\summary2.py:372: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
E:\Anaconda\Anaconda\lib\site-packages\statsmodels\iolib\summary2.py:374: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*'
E:\Anaconda\Anaconda\lib\site-packages\statsmodels\iolib\summary2.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  res.ix[:, 0][idx] = res.ix[:, 0][idx] + '*

In [53]:
Combined_Results.as_html

<bound method Summary.as_html of <class 'statsmodels.iolib.summary2.Summary'>
"""

                IAH BOS  BOS IAH   FLL JFK  JFK FLL   JFK LAX  LAX FLL 
-----------------------------------------------------------------------
BusyHour                 0.15**   1.05***   0.13**   0.04*     0.54*** 
                         (0.06)   (0.10)    (0.05)   (0.02)    (0.03)  
C(ClustID)[T.0] -4.56*** -4.14*** -18.04*** -8.03*** -2.69***  -1.85***
                (0.15)   (0.10)   (0.35)    (0.17)   (0.03)    (0.06)  
C(ClustID)[T.1] -2.22*** 0.58***  -12.14*** 1.42***  1.13***   -2.31***
                (0.15)   (0.16)   (0.37)    (0.30)   (0.07)    (0.05)  
C(ClustID)[T.2] -5.42*** -3.49*** -12.82*** -1.92*** -1.15***  -1.64***
                (0.16)   (0.11)   (0.59)    (0.19)   (0.04)    (0.09)  
C(ClustID)[T.3] -7.03*** -4.18***           2.21***  -1.08***  -2.31***
                (0.36)   (0.16)             (0.25)   (0.05)    (0.04)  
C(ClustID)[T.4] -5.14*** -0.40              5.11***  

In [54]:
(Combined_Results.tables[0]).to_csv(os.getcwd() + '/MNL/Estimation/All_OLS.csv')

In [294]:
for Ori in ['IAH','BOS','FLL','JFK','LAX']:
    for Des in ['IAH','BOS','FLL','JFK','LAX']:
        try:
            exec """pickle.dump(%s_%s, open(os.getcwd() + '/MNL/Estimation/OLS_' + '%s' + '_' + '%s', 'w'))"""%(Ori, Des, Ori, Des)
        except:
            pass

## Prediction Based on MNL and OLS

In [7]:
for Ori in ['IAH','BOS','FLL','JFK','LAX']:
    for Des in ['IAH','BOS','FLL','JFK','LAX']:
        try:
            exec """%s_%s = pickle.load(open(os.getcwd() + '/MNL/Estimation/OLS_' + '%s' + '_' + '%s.p', 'r'))"""%(Ori, Des, Ori, Des)
        except:
            pass

In [65]:
def Factor_Contribution_Predictor(Ori, Des, Factor):
    exec """MNL_DATA = %s_%s.MNL_Data"""%(Ori, Des)
    
    Fname_p = 'MNL_' + Ori + '_' + Des + '.p'
    MNL_Model = pickle.load(open(os.getcwd() + '/MNL/Estimation/' + Fname_p))
    
    # Construct a counter factual on MNL data
    MNL_DATA_No_Wx = MNL_DATA.copy()
    if Factor == 'convection':
        MNL_DATA_No_Wx.ix[:,2:18] = 0
    elif Factor == 'wind':
        MNL_DATA_No_Wx['Std_Wind_Dist'] = MNL_DATA_No_Wx['GC_Dist'] / 1000
        # 0
    elif Factor == 'MIT':
        MNL_DATA_No_Wx['MIT_Str'] = 0
#     elif Factor == 'all':
#         MNL_DATA_No_Wx.ix[:,2:18] = 0
#         MNL_DATA_No_Wx['MIT_Str'] = 0
#         MNL_DATA_No_Wx.ix[:,'Std_Wind_Dist'] = MNL_DATA_No_Wx.ix[:,'GC_Dist'] / 1000
        
    else:
        raise ValueError('Factor can only be convection, wind, MIT')
    
    # Prediction using MNL logit model
    MNL_DATA['Prob_With_Factor'] = MNL_Model.predict(MNL_DATA)
    MNL_DATA['Prob_No_Factor'] = MNL_Model.predict(MNL_DATA_No_Wx)
    
    MNL_Pred = MNL_DATA.groupby(['FID_x','Alt_id'])[['Prob_With_Factor','Prob_No_Factor']].mean().reset_index()
    MNL_Pred.ix[MNL_Pred.FID_x == 99999999,'Alt_id'] = -1
    MNL_Pred['OD_ClustID'] = Ori + '_' + Des + '_' + MNL_Pred.Alt_id.map(str)

    # Prediction using OLS
    exec """Mod2rslt = %s_%s.LR_Data1.ix[:,['FID_Member','Efficiency','OD_Clust','ClustID']]"""%(Ori, Des)
    exec """LR_No_Factor = %s_%s.LR_Data1.copy()"""%(Ori, Des)
    if Factor == 'convection':
        LR_No_Factor.ix[:,5:21] = 0
    elif Factor == 'MIT':
        LR_No_Factor['MIT_Str'] = 0
    else:
        pass
    exec """Mod2rslt['Pred_With_Factor'] = %s_%s.mod.predict(exog=%s_%s.LR_Data1)"""%(Ori, Des, Ori, Des)
    exec """Mod2rslt['Pred_No_Factor'] = %s_%s.mod.predict(exog=LR_No_Factor)"""%(Ori, Des)
    
    Mod2rslt = Mod2rslt.merge(MNL_Pred, left_on='OD_Clust', right_on='OD_ClustID', how='left')
    Mod2rslt['Pred_Factor_weight'] = Mod2rslt.Pred_With_Factor * Mod2rslt.Prob_With_Factor
    Mod2rslt['Pred_No_Factor_weight'] = Mod2rslt.Pred_No_Factor * Mod2rslt.Prob_No_Factor

    FactorContribution = Mod2rslt.groupby(['OD_Clust'])[['Pred_Factor_weight','Pred_No_Factor_weight']].mean().reset_index()
    Avg_Ineff = np.array(FactorContribution[['Pred_Factor_weight','Pred_No_Factor_weight']].sum())
#     Avg_Ineff = {}
#     Avg_Ineff[Ori + '_' + Des] = np.array(FactorContribution[['Pred_Factor_weight','Pred_No_Factor_weight']].sum().reset_index())
    
    return MNL_Pred[['OD_ClustID','Prob_With_Factor','Prob_No_Factor']], Mod2rslt, FactorContribution, Avg_Ineff

In [66]:
b = {}
bb = {}
bbb = {}
for Ori in ['IAH','BOS','FLL','JFK','LAX']:
    for Des in ['IAH','BOS','FLL','JFK','LAX']:
        try:
            _,_,_,a = Factor_Contribution_Predictor(Ori,Des, 'convection')
            _,_,_,aa = Factor_Contribution_Predictor(Ori,Des, 'wind')
            _,_,_,aaa = Factor_Contribution_Predictor(Ori,Des, 'MIT')
            b[Ori + '_' + Des] = a
            bb[Ori + '_' + Des] = aa
            bbb[Ori + '_' + Des] = aaa
        except:
            pass
Convection_Mat = pd.DataFrame(b).T
Convection_Mat.columns = ['With_Wx','No_Wx']

Wind_Mat = pd.DataFrame(bb).T
Wind_Mat.columns = ['With_Wind','No_Wind']

MIT_Mat = pd.DataFrame(bbb).T
MIT_Mat.columns = ['With_MIT','No_MIT']

In [67]:
Convection_Mat['Percentage_Change'] = (Convection_Mat.With_Wx - Convection_Mat.No_Wx)/Convection_Mat.With_Wx
Wind_Mat['Percentage_Change'] = (Wind_Mat.With_Wind - Wind_Mat.No_Wind)/Wind_Mat.With_Wind
MIT_Mat['Percentage_Change'] = (MIT_Mat.With_MIT - MIT_Mat.No_MIT)/MIT_Mat.With_MIT

In [72]:
Convection_Mat.to_csv(os.getcwd() + '/MNL/Estimation/Wx_Contributions.csv', index = True)
Wind_Mat.to_csv(os.getcwd() + '/MNL/Estimation/Wind_Contributions.csv', index = True)
MIT_Mat.to_csv(os.getcwd() + '/MNL/Estimation/MIT_Contributions.csv', index = True)

# Debug

In [101]:
i = 0
Airport = [['JFK','FLL'],['JFK','LAX'],['ORD','DCA'],['IAH','BOS'],
           ['FLL','JFK'],['LAX','JFK'],['DCA','ORD'],['BOS','IAH']]
for pair in Airport:
    i += 1
    if i == 1:
        PostMNL = PreProcessMNL(pair[0], pair[1], 2013)
        LR_Data = PostMNL.GetLR_Data()
    else:
        PostMNL = PreProcessMNL(pair[0], pair[1], 2013)
        LR_Data = LR_Data.append(PostMNL.GetLR_Data())
print(i)

8


E:\Anaconda\Anaconda\lib\site-packages\IPython\kernel\__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [102]:
LR_Data.to_csv('LR_EightPairs_New.csv',index = False)

### Regression Model I -- OD Fixed Effects

In [103]:
LR_Data = pd.read_csv(os.getcwd() + '/LR/LR_EightPairs_New.csv')
def GetSeason(x):
    if x >=12 or x<= 2:
        return 0 # Winter
    elif x <= 5:
        return 1 # Spring
    elif x <= 8:
        return 2 # Summer
    else:
        return 3 # Fall
LR_Data['season'] = LR_Data.LocalMonth.apply(lambda x: GetSeason(x))
LR_Data['BusyHour'] = LR_Data.LocalHour.apply(lambda x: (x>=8) & (x<=20)).astype(int)
LR_Data.Efficiency = LR_Data.Efficiency * 100

In [118]:
mod = sm.OLS.from_formula('Efficiency ~ TS_mean + Rain_mean + Squall_mean + OD_Clust + C(season) + BusyHour + Std_Wind_Dist', 
                          data=LR_Data[LR_Data.OD == 'IAH_BOS']).fit()
print mod.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                     214.5
Date:                Fri, 13 Jan 2017   Prob (F-statistic):               0.00
Time:                        16:44:31   Log-Likelihood:                -2841.5
No. Observations:                1664   AIC:                             5711.
Df Residuals:                    1650   BIC:                             5787.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                 7.96

In [269]:
Mod1rslt = LR_Data.ix[:,['OD','OD_Clust','ClustID','Efficiency']]
Pred_No_Wx = LR_Data.copy()
Pred_No_Wx.ix[:,4:20] = 0
Mod1rslt['Pred_Wx'] = mod.predict(exog=LR_Data)
Mod1rslt['Pred_No_Wx'] = mod.predict(exog=Pred_No_Wx)

In [212]:
Mod1rslt.groupby('OD')[['Pred_Wx','Pred_No_Wx']].mean()

,Pred_Wx,Pred_No_Wx
OD,,
BOS_IAH,2.580272,2.032670
DCA_ORD,4.195134,3.663963
FLL_JFK,3.585357,2.706356
IAH_BOS,4.365819,3.752653
JFK_FLL,3.051842,2.153418
JFK_LAX,2.046591,1.702839
LAX_JFK,2.425427,2.067796
ORD_DCA,4.105880,3.567358


In [253]:
Mod1rslt.mean()

ClustID       0.211723
Efficiency    3.102596
Pred_Wx       3.102596
Pred_No_Wx    2.586609
dtype: float64

### Regression Model II -- OD Cluster Fixed Effects

In [254]:
mod2 = sm.OLS.from_formula('Efficiency ~ TS_mean + Rain_mean + Squall_mean + OD_Clust + C(season) + BusyHour', data=LR_Data).fit()
print mod2.summary()

                            OLS Regression Results                            
Dep. Variable:             Efficiency   R-squared:                       0.589
Model:                            OLS   Adj. R-squared:                  0.588
Method:                 Least Squares   F-statistic:                     1328.
Date:                Thu, 22 Sep 2016   Prob (F-statistic):               0.00
Time:                        22:47:23   Log-Likelihood:            -1.0600e+05
No. Observations:               47359   AIC:                         2.121e+05
Df Residuals:                   47307   BIC:                         2.126e+05
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                  5

In [175]:
i = 0
N_clust = {}
temp = LR_Data.copy()
for gpidx, gp in temp.groupby('OD'):
    N_clust[gpidx] = gp.ClustID.max() + 2
    for clust in range(-1, N_clust[gpidx] - 1):
        i += 1
        gp.ix[:,'OD_Clust'] = gpidx + '_' + str(clust) 
        if i == 1:
            NewSet = gp.copy()
        else:
            NewSet = NewSet.append(gp)
NewSet = NewSet.reset_index(drop = 0)

Mod2rslt = NewSet.ix[:,['index','OD','OD_Clust','ClustID','Efficiency']]
Pred_No_Wx = NewSet.copy()
Pred_No_Wx.ix[:,5:21] = 0
Mod2rslt['Pred_Wx'] = mod2.predict(exog=NewSet)
Mod2rslt['Pred_No_Wx'] = mod2.predict(exog=Pred_No_Wx)

In [63]:
def PredictMNL(Ori, Des):
    Fname = 'MNL_' + Ori + '_' + Des + '_2013_Mean.csv'
    Fname_p = Ori + '_' + Des + '.p'
    MNL_DATA = pd.read_csv(os.getcwd() + '/MNL/' + Fname, header = 0)
    MNL_Model = pickle.load(open(os.getcwd() + '/MNL/' + Fname_p))
    
    MNL_DATA_No_Wx = MNL_DATA.copy()
    MNL_DATA_No_Wx.ix[:,2:18] = 0
    MNL_DATA['Prob_Wx'] = MNL_Model.predict(MNL_DATA)
    MNL_DATA['Prob_No_Wx'] = MNL_Model.predict(MNL_DATA_No_Wx)
    MNL_Pred = MNL_DATA.groupby(['FID_x','Alt_id'])[['Prob_Wx','Prob_No_Wx']].mean().reset_index()
    
    MNL_Pred.ix[MNL_Pred.FID_x == 99999999,'Alt_id'] = -1
    MNL_Pred['OD_ClustID'] = Ori + '_' + Des + '_' + MNL_Pred.Alt_id.map(str)
    return MNL_Pred[['OD_ClustID','Prob_Wx','Prob_No_Wx']]

In [74]:
i = 0
for pair in Mod2rslt.OD.unique():
    i += 1
    if i == 1:
        MNL_Pred = PredictMNL(pair[:3],pair[4:7])
    else:
        MNL_Pred = MNL_Pred.append(PredictMNL(pair[:3],pair[4:7]))
MNL_Pred = MNL_Pred.reset_index(drop = True)

In [262]:
Mod2rslt = Mod2rslt.merge(MNL_Pred, left_on='OD_Clust', right_on='OD_ClustID', how='left')
Mod2rslt['Pred_Wx_weight'] = Mod2rslt.Pred_Wx * Mod2rslt.Prob_Wx
Mod2rslt['Pred_No_Wx_weight'] = Mod2rslt.Pred_No_Wx * Mod2rslt.Prob_No_Wx

aa = Mod2rslt.groupby(['OD','index'])[['Pred_Wx_weight','Pred_No_Wx_weight']].sum().reset_index()
aa.groupby('OD')[['Pred_Wx_weight','Pred_No_Wx_weight']].mean()

,Pred_Wx_weight,Pred_No_Wx_weight
OD,,
BOS_IAH,2.580272,2.218133
DCA_ORD,4.195134,3.812454
FLL_JFK,3.585357,3.195368
IAH_BOS,4.365819,4.124445
JFK_FLL,3.051842,2.742559
JFK_LAX,2.046591,1.899419
LAX_JFK,2.425427,2.260838
ORD_DCA,4.105880,3.725363


In [263]:
aa[['Pred_Wx_weight','Pred_No_Wx_weight']].mean()

Pred_Wx_weight       3.102596
Pred_No_Wx_weight    2.833009
dtype: float64

In [270]:
aa['Wx_Compare'] = mod2.predict(exog = LR_Data)
aa['No_Wx_Compare'] = mod2.predict(exog = Pred_No_Wx)
# Pred_No_Wx

In [272]:
aa.No_Wx_Compare.mean()

2.995568076241766

## Another approach: Drop all outlier groups

In [ ]:
modelset = MNL_DATA[~((MNL_DATA.FID_x == 99999999)&(MNL_DATA.CHOICE == 1))]
modelset = modelset.dropna(subset=['MeanWindSpeed', 'Wind_Dist'], how='all').reset_index(drop = 1)
print(modelset.FID_Member.unique().shape[0])
modelset['Wind_Dist'] = modelset['Wind_Dist'] / 1000

BaseSpec = collections.OrderedDict()
BaseName = collections.OrderedDict()

BaseSpec['intercept'] = [0,1,2,3]
BaseName['intercept'] = ['ASC_R0','ASC_R1','ASC_R2','ASC_R3']

BaseSpec['Morning'] = [0,1,2,3]
BaseName['Morning'] = ['MorningFlight_R0','MorningFlight_R1','MorningFlight_R2','MorningFlight_R3']

BaseSpec['Spring'] = [0,1,2,3]
BaseName['Spring'] = ['Spring_R0','Spring_R1','Spring_R2','Spring_R3']

BaseSpec['Summer'] = [0,1,2,3]
BaseName['Summer'] = ['Summer_R0','Summer_R1','Summer_R2','Summer_R3']

BaseSpec['Fall'] = [0,1,2]
BaseName['Fall'] = ['Fall_R0','Fall_R1','Fall_R2']
    
BaseSpec['TS_mean'] = [[0,1,2,3]]
BaseName['TS_mean'] = ['Thunderstorm']

BaseSpec['Rain_mean'] = [[0,1,2,3]]
BaseName['Rain_mean'] = ['Rain']

# BaseSpec['MeanWindSpeed'] = [[0,1,2,3]]
# BaseName['MeanWindSpeed'] = ['MeanWindSpeed']

BaseSpec['Wind_Dist'] = [[0,1,2,3]]
BaseName['Wind_Dist'] = ['Wind_Dist']

N_var = 0
for key in BaseSpec.keys():
    N_var += len(BaseSpec[key])
    
MNL_Model = pylogit.create_choice_model(data = modelset, alt_id_col = 'Alt_id',obs_id_col = 'FID_Member',
                                        choice_col = 'CHOICE',specification = BaseSpec, model_type = 'MNL', names = BaseName)
MNL_Model.fit_mle(np.zeros(N_var))
Result1 = MNL_Model.get_statsmodels_summary()
Result1.as_csv